In [1]:
# Install necessary libraries
!pip install -q transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/phi-2"  # Lightweight model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [23]:
from google.colab import files
uploaded = files.upload()


Saving my_dataset_40.json to my_dataset_40.json


In [26]:
from datasets import Dataset
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json("my_dataset_40.json")

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [38]:
def tokenize(example):
    text = example["prompt"] + "\n" + example["response"]
    result = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors=None
    )
    result["labels"] = result["input_ids"].copy()  # For causal LM
    return result

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [39]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./my-custom-model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=1,
    save_steps=5,
    warmup_steps=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    remove_unused_columns=False,   # important for PEFT models
    report_to=[]                   # disable wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-39-1738046468.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,0.512500
2,0.573600
3,0.612700
4,0.545100
5,0.546700
6,0.465900
7,0.467100
8,0.557100
9,0.492900
10,0.483800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=60, training_loss=0.36323955555756887, metrics={'train_runtime': 64.2371, 'train_samples_per_second': 1.868, 'train_steps_per_second': 0.934, 'total_flos': 488675318169600.0, 'train_loss': 0.36323955555756887, 'epoch': 3.0})

In [40]:
prompt = "### Prompt:\nWhat inspires you to keep learning?\n\n### Response:\n"


In [41]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

output = pipe(prompt, max_new_tokens=100, temperature=0.7, do_sample=True)
print(output[0]["generated_text"])


Device set to use cuda:0
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible wi

### Prompt:
What inspires you to keep learning?

### Response:
I have in


In [47]:
# ✅ Step 1: Load the already-uploaded dataset
import pandas as pd
from datasets import Dataset

df = pd.read_json("/content/my_dataset.json")  # File already uploaded
dataset = Dataset.from_pandas(df)

# ✅ Step 2: Fix tokenizer padding & tokenize the dataset
tokenizer.pad_token = tokenizer.eos_token  # Necessary for padding

def tokenize(example):
    text = example["prompt"] + "\n" + example["response"]
    result = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Prepare model for PEFT
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "fc1", "fc2"],  # adjust based on model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Add LoRA adapters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


# ✅ Step 3: Define training arguments and trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./my-custom-model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=1,
    save_steps=5,
    warmup_steps=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# ✅ Step 4: Train the model
trainer.train()


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/tmp/ipython-input-47-3911196317.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 10,485,760 || all params: 2,790,169,600 || trainable%: 0.3758


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Step,Training Loss
1,8.434500
2,8.791000
3,9.258300
4,9.343000
5,8.872600
6,9.100800
7,8.993400
8,8.836600
9,8.182500
10,8.475700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=15, training_loss=8.546352354685466, metrics={'train_runtime': 20.1551, 'train_samples_per_second': 1.488, 'train_steps_per_second': 0.744, 'total_flos': 122531217408000.0, 'train_loss': 8.546352354685466, 'epoch': 3.0})

In [48]:

# ✅ Step 7: Inference (Generate Answer)
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "### Prompt:\nWhat inspires you to keep learning?\n\n### Response:\n"

output = pipe(prompt, max_new_tokens=100, temperature=0.7, do_sample=True)
generated = output[0]["generated_text"].split("### Response:\n")[-1]

print(f"\n🧠 Prompt: What inspires you to keep learning?\n📝 Response: {generated.strip()}")


Device set to use cuda:0
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible wi


🧠 Prompt: What inspires you to keep learning?
📝 Response: I’s in the list of the room, and in the number of the second-m and more than just like a different cultures.





    return s = self-in-up exercises.

The US-world scenarios. This could be a series of the most of the power of the future.
        #        answer.


In conclusion, and
In the car accident, as it also known for the
